In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
phylake1337_fire_dataset_path = kagglehub.dataset_download('phylake1337/fire-dataset')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def extract_features(image_paths, model):
    features = []
    for img_path in image_paths:
        img_array = preprocess_image(img_path)
        feature = model.predict(img_array)
        features.append(feature.flatten())
    return np.array(features)

fire_image_dir = "/kaggle/input/fire-dataset/fire_dataset/fire_images"
non_fire_image_dir = "/kaggle/input/fire-dataset/fire_dataset/non_fire_images"

fire_images = [os.path.join(fire_image_dir, f) for f in os.listdir(fire_image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
non_fire_images = [os.path.join(non_fire_image_dir, f) for f in os.listdir(non_fire_image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

fire_features_resnet = extract_features(fire_images, resnet_model)
non_fire_features_resnet = extract_features(non_fire_images, resnet_model)

X = np.vstack([fire_features_resnet, non_fire_features_resnet])
y = np.array([1] * len(fire_images) + [0] * len(non_fire_images))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)

gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_classifier.fit(X_train, y_train)
y_pred_gb = gb_classifier.predict(X_test)

base_learners = [
    ('svm', SVC(probability=True, random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42))
]

stacking_classifier = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
stacking_classifier.fit(X_train, y_train)
y_pred_stack = stacking_classifier.predict(X_test)

voting_classifier_hard = VotingClassifier(estimators=[('rf', rf_classifier), ('gb', gb_classifier), ('stack', stacking_classifier)], voting='hard')
voting_classifier_hard.fit(X_train, y_train)
y_pred_hard = voting_classifier_hard.predict(X_test)

voting_classifier_soft = VotingClassifier(estimators=[('rf', rf_classifier), ('gb', gb_classifier), ('stack', stacking_classifier)], voting='soft')
voting_classifier_soft.fit(X_train, y_train)
y_pred_soft = voting_classifier_soft.predict(X_test)

print(f"Bagging (Random Forest) Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Boosting (Gradient Boosting) Accuracy: {accuracy_score(y_test, y_pred_gb):.4f}")
print(f"Stacking Classifier Accuracy: {accuracy_score(y_test, y_pred_stack):.4f}")
print(f"Hard Voting Accuracy: {accuracy_score(y_test, y_pred_hard):.4f}")
print(f"Soft Voting Accuracy: {accuracy_score(y_test, y_pred_soft):.4f}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50

# Function to load and preprocess a single image
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to extract features from a single image using the pretrained ResNet50 model
def extract_image_features(img_path, model):
    img_array = preprocess_image(img_path)
    feature = model.predict(img_array)
    return feature.flatten()

# Path to the image you want to predict
img_path = "/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.100.png"  # Update with the path to the image

# Load ResNet50 model (same model used for feature extraction)
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Extract features for the input image
img_feature = extract_image_features(img_path, resnet_model)

# Reshape the image features to match the model input dimensions
img_feature = np.expand_dims(img_feature, axis=0)

# --- Making predictions with the trained models ---
# Hard Voting Prediction
hard_voting_prediction = voting_classifier_hard.predict(img_feature)
print(f"Hard Voting Prediction (Fire=1, Non-Fire=0): {hard_voting_prediction[0]}")

# Soft Voting Prediction
soft_voting_prediction = voting_classifier_soft.predict(img_feature)
print(f"Soft Voting Prediction (Fire=1, Non-Fire=0): {soft_voting_prediction[0]}")

# Bagging (Random Forest) Prediction
rf_prediction = rf_classifier.predict(img_feature)
print(f"Random Forest Prediction (Fire=1, Non-Fire=0): {rf_prediction[0]}")

# Boosting (Gradient Boosting) Prediction
gb_prediction = gb_classifier.predict(img_feature)
print(f"Gradient Boosting Prediction (Fire=1, Non-Fire=0): {gb_prediction[0]}")

# Stacking Classifier Prediction
stacking_prediction = stacking_classifier.predict(img_feature)
print(f"Stacking Classifier Prediction (Fire=1, Non-Fire=0): {stacking_prediction[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Hard Voting Prediction (Fire=1, Non-Fire=0): 0
Soft Voting Prediction (Fire=1, Non-Fire=0): 0
Random Forest Prediction (Fire=1, Non-Fire=0): 0
Gradient Boosting Prediction (Fire=1, Non-Fire=0): 0
Stacking Classifier Prediction (Fire=1, Non-Fire=0): 0
